# Endless Runner Q-table training

Heavily inspired on the Q-table Colab notebook from Huggingface's DeepRL course: https://huggingface.co/learn/deep-rl-course/unit2/hands-on

(Aquí dejo esto para que no se me olvide. Para correr el ambiente, primero se debe instalar el paquete de GameEnvs, para hacer esto lo que hago es irme a /GameEnvs y corro pip install -e .)

## Import the packages

In [2]:
pip install -e GameEnvs

Obtaining file:///D:/Repository/GameDevsCSF/RL-environments/GameEnvs
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: GameEnvs
    Found existing installation: GameEnvs 0.0.1
    Uninstalling GameEnvs-0.0.1:
      Successfully uninstalled GameEnvs-0.0.1
  Running setup.py develop for GameEnvs
Note: you may need to restart the kernel to use updated packages.


By running the next cell, we'll kill the kernel to make sure the installed package is imported correctly.

In [3]:
# Restart the kernel after running this cell to make sure that the package is imported correctly.
import os
os._exit(00)

: 

Now we can run the next cell by stopping it, restarting the kernel and running it again.

In [2]:
import gymnasium as gym
import GameEnvs

## Create the environment

In [3]:
env = gym.make("EndlessRunner-v0")

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


## Understanding our environment

In [4]:
print("Action space:", env.action_space)
print("Sample action:", env.action_space.sample())

Action space: Discrete(3)
Sample action: 1


The actions are: do nothing, jump or duck

## Initialize the Q-table
Need to discretize the observations to make the Q-table. The solution seems to be to bin the observations, the question is how.

We have the obstacle's x position, the obstacles y positon and the player's y positon.

- Obstacle x: Goes from being at x = 800 to x = 50 (behind the player). The bin size could translate into "how fast" the agent reacts to having an obstacle in front of it.
- Obstacle y: is either on the floor (0) or above the floor (1).
- Player y: Is either standing (0), ducking (1), jumping (2).

In [5]:
print("Observation space:", env.observation_space)
print("Sample observation:", env.observation_space.sample())

Observation space: Dict('obstacle_height': Discrete(2), 'obstacle_x': Box(50.0, 800.0, (1,), float32), 'player_state': Discrete(3))
Sample observation: OrderedDict({'obstacle_height': 0, 'obstacle_x': array([246.84254], dtype=float32), 'player_state': 1})


In [6]:
for i in range(10):
    env.reset()
    for t in range(10):
        action = env.action_space.sample()
        observation, reward, terminated, truncated, info = env.step(action)
        if terminated:
            print("Episode finished after {} timesteps".format(t+1))
            break

c:\Users\Prod\miniconda3\envs\rl-lab-dev\Lib\site-packages\gymnasium\utils\passive_env_checker.py:137: UserWarning: WARN: The obs returned by the `reset()` method was expecting a numpy array, actual type: <class 'numpy.int64'>
  logger.warn(
c:\Users\Prod\miniconda3\envs\rl-lab-dev\Lib\site-packages\gymnasium\spaces\box.py:227: UserWarning: WARN: Casting input x to numpy array.
  logger.warn("Casting input x to numpy array.")
c:\Users\Prod\miniconda3\envs\rl-lab-dev\Lib\site-packages\gymnasium\utils\passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\Prod\miniconda3\envs\rl-lab-dev\Lib\site-packages\gymnasium\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
c:\Users\Prod\miniconda3\envs\rl-lab-dev\Lib\site-packages\gy

: 